In [68]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [95]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('h1', class_='content--title').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def get_link_nogenre(start=0):
    notyet = []
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    for dic in data:
        if dic['genre'] == []:
            notyet.append(dic['url'])
    return notyet  

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [96]:
htmls = get_link(0); htmls

['https://www3.nhk.or.jp/news/html/20190120/k10011784611000.html',
 'https://www3.nhk.or.jp/news/html/20191010/k10012120321000.html',
 'https://www3.nhk.or.jp/news/html/20191017/k10012135411000.html']

In [99]:
htmls = get_link_nogenre(); htmls[:100]

['https://www3.nhk.or.jp/news/html/20190117/k10011780361000.html',
 'https://www3.nhk.or.jp/news/html/20190123/k10011787281000.html',
 'https://www3.nhk.or.jp/news/html/20190123/k10011788211000.html',
 'https://www3.nhk.or.jp/news/html/20190205/k10011803571000.html',
 'https://www3.nhk.or.jp/news/html/20190221/k10011823181000.html',
 'https://www3.nhk.or.jp/news/html/20190226/k10011828651000.html',
 'https://www3.nhk.or.jp/news/html/20190226/k10011828871000.html',
 'https://www3.nhk.or.jp/news/html/20190307/k10011838501000.html',
 'https://www3.nhk.or.jp/news/html/20190311/k10011842891000.html',
 'https://www3.nhk.or.jp/news/html/20190311/k10011844261000.html',
 'https://www3.nhk.or.jp/news/html/20190313/k10011847001000.html',
 'https://www3.nhk.or.jp/news/html/20190314/k10011847471000.html',
 'https://www3.nhk.or.jp/news/html/20190314/k10011848051000.html',
 'https://www3.nhk.or.jp/news/html/20190315/k10011848721000.html',
 'https://www3.nhk.or.jp/news/html/20190315/k10011848861000.ht

# scrape

In [50]:
driver = webdriver.Chrome()

In [100]:
for nhkurl in tqdm.tqdm(htmls[1:-2]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(20)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)







  0%|          | 0/833 [00:00<?, ?it/s]





  0%|          | 1/833 [00:57<13:13:16, 57.21s/it]





  0%|          | 2/833 [01:56<13:22:46, 57.96s/it]





  0%|          | 3/833 [03:01<13:48:46, 59.91s/it]





  0%|          | 4/833 [04:01<13:49:52, 60.06s/it]





  1%|          | 5/833 [05:09<14:20:43, 62.37s/it]





  1%|          | 6/833 [06:15<14:35:10, 63.50s/it]





  1%|          | 7/833 [07:16<14:24:23, 62.79s/it]





  1%|          | 8/833 [08:30<15:08:02, 66.04s/it]





  1%|          | 9/833 [09:36<15:07:27, 66.08s/it]





  1%|          | 10/833 [10:34<14:34:19, 63.74s/it]





  1%|▏         | 11/833 [11:40<14:41:35, 64.35s/it]





  1%|▏         | 12/833 [12:42<14:28:25, 63.47s/it]





  2%|▏         | 13/833 [13:51<14:51:03, 65.20s/it]





  2%|▏         | 14/833 [14:37<13:32:44, 59.54s/it]





  2%|▏         | 15/833 [15:29<12:58:21, 57.09s/it]





  2%|▏         | 16/833 [16:25<12:53:06, 56.78s/it]





  2%|▏         | 17/833 [17:16<12:31:54, 55.29

In [94]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
dic = scrape(html)
print(dic)
js(dic)

{'id': 'k10011846961000', 'title': '買い物弱者など解決へ ロボットがコンビニの商品配達する実験', 'article': '高齢化に伴う「買い物弱者」などの解決につなげようと、自動で動くロボットがコンビニの商品を配達する実験が神奈川県の大学で行われました。\nこれは大手コンビニチェーンの「ローソン」と自動運転技術を開発するベンチャー企業の「ＺＭＰ」などが行いました。実験は大学内で行われ、利用者がスマートフォンの専用のアプリを使って商品を注文すると、高さ、幅などが１メートルに満たない箱型のロボットが、仮設の店舗から商品を届けます。センサーやカメラが周りの状況を認識しているということで、人が近づくと自動的に停止するほか、人とすれ違った際には音声であいさつもしてくれます。両社では、こうしたロボットで高齢化に伴う買い物弱者やドライバーの人手不足といった課題の解決につなげたい考えです。また公道を移動できるよう国に対して法整備を働きかけていくことにしています。ローソンの牧野国嗣オープン・イノベーションセンター長は「時間がなかったり、体が不自由で店に行けなかったりするなど、さまざまなニーズに応えるため、実用化を進めていきたい」と話していました。', 'genre': ['ビジネス', '社会'], 'keywords': [], 'url': 'https://www3.nhk.or.jp/news/html/20190313/k10011846961000.html', 'datePublished': '2019-03-13T17:35:18+09:00', 'dateModified': '2019-03-27T14:00:23+09:00'}


In [91]:
html = str(driver.page_source.encode('utf-8'))
re.search(r'"genre":(.+?),', html)

# clean category & keyword

In [75]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))

4712


In [79]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

4712


[('社会', 1232),
 ('国際', 1039),
 ('ビジネス', 603),
 ('スポーツ', 532),
 ('政治', 518),
 ('科学・文化', 504),
 ('暮らし', 285),
 ('地域', 234),
 ('気象・災害', 209)]

In [77]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        j = re.sub('災害|気象', '気象・災害', j)
        if j in category:
            newgenre.append(j)
        else:
            newkey.append(j)
    for j in dic['keywords']:
        j = re.sub('災害|気象', '気象・災害', j)
        if j in category:
            newgenre.append(j)
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    